In [1]:
!pip install pandas

In [2]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium --yes
import folium

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\james\Anaconda3

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.0.1               |           py37_0         722 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.8.3               |             py_0          54 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         829 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/win-64::altair-3.0.1-py37_0
  branca             conda-forge/noarch::branca-0.3.1-py_0
  folium             conda-forge/noarch::folium-0.8.3-py_0
  vincent            conda-forge/noa

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/ninjananjo/Clothing-Store-Competition/master/StoreLocations.csv")
df.shape

(8, 56)

In [4]:
df = df[['TOWN','NAME','ID','COUNTRY','Address1','Postcode','Host']]
#df = df[df['ID'].str.startswith("TS")]
df.head(10)

,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Host
0,LONDON (OXFORD CIRCUS),LONDON OXFORD CIRCUS,MS0325,ENG,"Ground Floor, 36-38 Great Castle Street",W1W 8LG,TS0001
1,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TM7329,ENG,214 Oxford Street,W1W 8LG,TS0001
2,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,TS0001
3,LONDON (CITY),CHEAPSIDE,TS0873,ENG,Unit Msu1,EC2V 6AH,TS0873
4,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE OUTFIT,OU0207,ENG,72-76,NE1 7DF,OU0207
5,Z USA LOS ANGELES,LOS ANGELES,TM8104,US,189 The Grove Drive,90036,TS1271
6,MANCHESTER (TRAFFORD C),TRAFFORD CENTRE,TS1012,ENG,130-134 Regent Crescent,M17 8AF,TS1012
7,HAYES,HAYES LOMBARDY RET PARK OUTFIT,OU0201,ENG,Unit 3,UB3 3EX,OU0201


In [13]:
def getLocation(storeName, postcode, country):
    
    if country == 'ENG': country = "UK"
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(postcode+" "+country)
    location_alt = geolocator.geocode(storeName+" "+country)
    try:
        lat = location.latitude
        lng = location.longitude
    except:
        try:
            lat = location_alt.latitude
            lng = location_alt.longitude
        except:
            lat = 0
            lng = 0
            
    return(lat,lng)

In [16]:
latitude = []
longitude = []

for storeName, postcode, country in zip(df['NAME'],df['Postcode'],df['COUNTRY']):
    lati, lngi = getLocation(storeName, postcode, country)
    latitude.append(lati)
    longitude.append(lngi)

df.loc[:,'Latitude']= latitude
df.loc[:,'Longitude']= longitude

,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Host,Latitude,Longitude
0,LONDON (OXFORD CIRCUS),LONDON OXFORD CIRCUS,MS0325,ENG,"Ground Floor, 36-38 Great Castle Street",W1W 8LG,TS0001,51.515534,-0.141352
1,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TM7329,ENG,214 Oxford Street,W1W 8LG,TS0001,51.515534,-0.141352
2,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,TS0001,51.515534,-0.141352
3,LONDON (CITY),CHEAPSIDE,TS0873,ENG,Unit Msu1,EC2V 6AH,TS0873,51.514098,-0.094914
4,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE OUTFIT,OU0207,ENG,72-76,NE1 7DF,OU0207,54.975903,-1.612382
5,Z USA LOS ANGELES,LOS ANGELES,TM8104,US,189 The Grove Drive,90036,TS1271,34.067233,-118.352108
6,MANCHESTER (TRAFFORD C),TRAFFORD CENTRE,TS1012,ENG,130-134 Regent Crescent,M17 8AF,TS1012,53.462559,-2.346119
7,HAYES,HAYES LOMBARDY RET PARK OUTFIT,OU0201,ENG,Unit 3,UB3 3EX,OU0201,51.516683,-0.405498


In [17]:
#remove regions that location information was not available
df = df[(df['Latitude']!=0)| (df['Longitude']!=0)]
print(df.shape[0])
df

8


,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Host,Latitude,Longitude
0,LONDON (OXFORD CIRCUS),LONDON OXFORD CIRCUS,MS0325,ENG,"Ground Floor, 36-38 Great Castle Street",W1W 8LG,TS0001,51.515534,-0.141352
1,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TM7329,ENG,214 Oxford Street,W1W 8LG,TS0001,51.515534,-0.141352
2,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,TS0001,51.515534,-0.141352
3,LONDON (CITY),CHEAPSIDE,TS0873,ENG,Unit Msu1,EC2V 6AH,TS0873,51.514098,-0.094914
4,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE OUTFIT,OU0207,ENG,72-76,NE1 7DF,OU0207,54.975903,-1.612382
5,Z USA LOS ANGELES,LOS ANGELES,TM8104,US,189 The Grove Drive,90036,TS1271,34.067233,-118.352108
6,MANCHESTER (TRAFFORD C),TRAFFORD CENTRE,TS1012,ENG,130-134 Regent Crescent,M17 8AF,TS1012,53.462559,-2.346119
7,HAYES,HAYES LOMBARDY RET PARK OUTFIT,OU0201,ENG,Unit 3,UB3 3EX,OU0201,51.516683,-0.405498


In [18]:
CLIENT_ID = 'QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO' # your Foursquare ID
CLIENT_SECRET = '1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH' # your Foursquare Secret
VERSION = '20190501' # Foursquare API version
LIMIT = 400 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius
categoryId = '4bf58dd8d48988d1f2941735'

In [114]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    
    store_list = []
    lat_list = []
    lng_list = []
    
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(    
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
        
        #print(url) 
        
        # make the GET request
        results = requests.get(url).json()["response"]
        
        #flatten JSON file
        resultsVen = json_normalize(results['venues'])
        resultsCat = json_normalize(data=results['venues'], record_path=['categories'], record_prefix='cat_', errors='ignore')
        resultsVenCat = pd.concat([resultsVen, resultsCat], axis=1)
        resultsVenCat = resultsVenCat[['id','name','location.distance','location.formattedAddress','location.lat','location.lng','cat_name']]
        
        #Add region name 
        for i in range(0,resultsVen.shape[0]):
            store_list.append(name)
            lat_list.append(lat)
            lng_list.append(lng)

        store = pd.DataFrame([store_list,lat_list,lng_list]).T
        nearbyVenues = pd.concat([store, resultsVenCat], axis=1)

    return(nearbyVenues)

In [115]:
regions_venues = pd.DataFrame()

#request venue information for each region and store in regions_venues
for storeId in df['ID'].unique():
    store_info = df[df['ID']==storeId]
    regions_venues = regions_venues.append(getNearbyVenues(names=store_info['ID'],
                                                  latitudes=store_info['Latitude'],
                                                  longitudes=store_info['Longitude'],
                                                  radius=radius),ignore_index=True)
    
#rename columns
regions_venues.rename(columns={'id':'Venue ID',
                          'name':'Venue Name',
                          'location.distance':'Venue Distance',
                          'location.formattedAddress':'Venue Address',
                          'location.lat':'Venue Lat',
                          'location.lng':'Venue Lng',
                          'cat_name':'Venue Category'}, inplace=True)

print(regions_venues.shape[0])

246


In [116]:
regions_venues

,0,1,2,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
0,MS0325,51.5155,-0.141352,4ac518f0f964a520afad20e3,NikeTown,33,"[236 Oxford St, London, Greater London, W1C 1D...",51.515488,-0.141824,Sporting Goods Shop
1,MS0325,51.5155,-0.141352,4ac518f1f964a520ecad20e3,JD Sports,127,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
2,MS0325,51.5155,-0.141352,5c7ea98b07190200259a8672,Lacoste,1291,"[6 James St, Covent Garden, London, Greater Lo...",51.512678,-0.123285,Clothing Store
3,MS0325,51.5155,-0.141352,51d86b1d498e44075e0a7840,Foot Locker,439,"[109 Oxford Street, London, Greater London, W1...",51.515890,-0.135038,Sporting Goods Shop
4,MS0325,51.5155,-0.141352,5ca5b84a9d7468003917baa1,Up & Running,2405,"[72-74 Fleet Street, London, Greater London, E...",51.514142,-0.106705,Sporting Goods Shop
5,MS0325,51.5155,-0.141352,5804fc2b38fa33f7c3387dae,New Balance,175,"[287 - 291 Oxford Street, London, Greater Lond...",51.515116,-0.143791,Sporting Goods Shop
6,MS0325,51.5155,-0.141352,4da064b3784f3704428698af,Sports Direct,246,"[150 Oxford St, London, Greater London, W1D 1N...",51.515899,-0.137837,Sporting Goods Shop
7,MS0325,51.5155,-0.141352,564e07e5498e190a55f52c35,Descente UK,238,"[Fouberts Place (Carmaby Street), London, Grea...",51.513720,-0.139531,Sporting Goods Shop
8,MS0325,51.5155,-0.141352,4c5ecca4d25320a1b698c97a,Runners Need,193,"[42-48 Great Portland St, London, Greater Lond...",51.517268,-0.141436,Sporting Goods Shop
9,MS0325,51.5155,-0.141352,5b9e565301bc5a002ca235fa,Foot Locker,1214,"[542-546 Oxford Street, London, Greater London...",51.513867,-0.158683,Sporting Goods Shop


In [12]:
jdsports = regions_venues[regions_venues['Venue Name']=='JD Sports']
jdsports.to_csv('jdsports_proximty.csv')
jdsports

,Store ID,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
2,TS0001,4ac518f1f964a520ecad20e3,JD Sports,127,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
13,TS0001,5895ef8201f0776e38be9d23,JD Sports,579,"[385-389 Oxford St, Mayfair, Greater London, U...",51.514111,-0.149394,Sporting Goods Shop
58,TS0873,4ccc1f952dc4370440aac408,JD Sports,2331,"[55 Chapel Street Market, Islington, Greater L...",51.533512,-0.107551,Sporting Goods Shop
95,TS1012,4edd380cbe7be2833b142d45,JD Sports,482,"[48 Regents Crescent, Unit L12, Trafford Centr...",53.466613,-2.348693,Sporting Goods Shop


In [ ]:
#Which TM/TS stores do not have a JD Sports withing a 2km radius?
#Add map feature with TM/TS as centre point and surrounding JD Sports.